In [3]:
import pandas as pd
import numpy as np
import nltk

Подгрузим train датасет и проверим сразу дизбаланс классов.

In [5]:
df = pd.read_csv('train_spam.csv')
df['text_type'].value_counts()


text_type
ham     11469
spam     4809
Name: count, dtype: int64

Дизбаланс классов присутствует, но не большой. Изучим набор представленный набор данных: посмотрим на самые популярные символы, которые встречаются в текстах спама и не спама.

In [7]:
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer

spam_messages = df[df['text_type'] == 'spam']['text']
not_spam_messages = df[df['text_type'] == 'ham']['text']

tw = TweetTokenizer()

spam = []
not_spam = []

for msg in spam_messages:
  spam += tw.tokenize(msg)

for msg in not_spam_messages:
  not_spam += tw.tokenize(msg)

stop_words = set(stopwords.words('english'))
spam  = [word for word in spam if word not in stop_words]
not_spam = [word for word in not_spam if word not in stop_words]

total_spam_words = len(spam)
total_not_spam_words = len(not_spam)

stem = SnowballStemmer("english")
stem_spam = [stem.stem(w) for w in spam]
stem_not_spam = [stem.stem(w) for w in not_spam]

spam_counter = Counter(stem_spam)
not_spam_counter = Counter(stem_not_spam)
spam_df = pd.DataFrame()
not_spam_df = pd.DataFrame()
feature1=[]
feature2=[]

print('Top 20 spam words: ')
for word, count in spam_counter.most_common(20):
    print(word, ": ", count)
    feature1.append(word)
print('\nTop 20 not spam words: ')
for word, count in not_spam_counter.most_common(20):
    print(word, ": ", count)
    feature2.append(word)


Top 20 spam words: 
👇 :  2187
free :  1085
get :  966
1635465 :  827
1 :  799
️ :  772
$ :  759
/ :  690
call :  653
invest :  648
2 :  648
! :  622
receiv :  595
click :  574
new :  562
link :  546
: :  529
✅ :  526
offer :  524
make :  521

Top 20 not spam words: 
ect :  5344
enron :  4850
vinc :  4530
1635465 :  4023
url :  3625
1635465 1635 :  3131
hou :  2660
kaminski :  2382
2000 :  2204
subject :  2117
pleas :  1948
j :  1929
cc :  1909
pm :  1895
thank :  1886
com :  1854
would :  1850
465 1635465 :  1790
get :  1632
time :  1613


Видны хорошие признаки спама - специальные смайлики, они дают хорошие шансы, что сообщение спам. Но наблюдается также и загрязнение в данных. Число 1635465 встречается под обеими метками, и абсолютно не понятно к чему оно - нужно очистить данные от этого. Так же слова вроде get встречаются и там, и там.
Удалим лишние признаки.

In [ ]:
delete1 = [3,3,3,3,3,5,5,9]
delete2 = [3,4,6,13,-3]
for i in delete1:
  f1.pop(i)
for i in delete2:
  f2.pop(i)

Помимо самого текста будем использовать признаки общие характеристики текста:
1. Подсчет слов в тексте, схожих на слова спама.
2. Подсчет слов в тексте, схожих на слова не спама.
3. Подсчет специальных знаков.
4. Подсчет количества цифр в тексте
5. Длина текста.
6. Количество слов в тексте.
7. Количество смайликов в тексте.

In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.0 MB/s eta 0:00:00


Разобъем текст на токены, избавим его от стоп слов и некоторых грязных данных, а также воспользуемся стеммингом над английским текстом.

In [ ]:
def PrepareData(text):
  tw = TweetTokenizer()
  text = tw.tokenize(text)
  stop_words = set(stopwords.words('english')) | set(['1635465','465','1635','465 1635465','1635465 1635'])
  text  = [word for word in text if word not in stop_words]
  stem = SnowballStemmer("english")
  stem_text = [stem.stem(w) for w in text]
  return ' '.join(stem_text)
test = pd.read_csv('test_spam.csv')
df['text'] = df['text'].apply(PrepareData)
test['text'] = test['text'].apply(PrepareData)

Обозначим символы и слова, которые мы будем считать как признак текста.

In [ ]:
add1 = ['money','earn','best','business']
feature3 = ['$','&','@','!','?','.','#','№',':',';']
feature4 = ['0','1','2','3','4','5','6','7','8','9']
feature1 = feature1 + add1

Добавим ранее объявленные признаки в train и test файлы.

In [ ]:
import emoji
test_train = [df,test]
for i in test_train:
  i['spam_symbols'] = i['text'].apply(lambda text: sum(word in text for word in feature1))
  i['not_spam_symbols'] = i['text'].apply(lambda text: sum(word in text for word in feature2))
  i['special_symbols'] = i['text'].apply(lambda text: sum(word in text for word in feature3))
  i['digits'] = i['text'].apply(lambda text: sum(word in text for word in feature4))
  i['text_len'] = i['text'].apply(lambda text: len(text))
  i['words_count'] = i['text'].apply(lambda text: len(text.split(' ')))
  i['emojis_count'] = i['text'].apply(lambda text: sum(emoji.is_emoji(word) for word in text))

Сохраняем полученные данные.

In [ ]:
df.to_csv('prepared_train.csv',index = False)
test.to_csv('prepared_test.csv',index = False)


In [ ]:
df

,text_type,text,spam_symbols,not_spam_symbols,special_symbols,digits,text_len,words_count,emojis_count
0,ham,make sure alex know birthday fifteen minut far...,1,0,0,0,59,10,0
1,ham,resum john lavorato thank vinc get move right ...,3,11,0,4,362,64,0
2,spam,plzz visit websit moviesgodml get movi free al...,3,2,0,0,96,18,4
3,spam,urgent mobil number award £ 2000 prize guarant...,1,1,0,8,104,17,0
4,ham,overview hr associ analyst project per david r...,0,8,0,4,589,99,0
...,...,...,...,...,...,...,...,...,...
16273,spam,interest binari option trade may continu infor...,0,1,0,6,72,12,0
16274,spam,dirti pictureblyk aircel thank valu member her...,1,3,1,1,326,51,0
16275,ham,could g mon sep david ree wrote mon sep pm rob...,2,4,0,0,442,78,0
16276,ham,insta reel par 80 गंद bhara pada hai 👀 kuch bh...,0,0,0,2,90,20,2
